In [1]:
import numpy as np
import sympy

sympy.init_printing()

In [2]:
A, B, C, D, E, F, G, H = sympy.symbols("A B C D E F G H")

Get a pool of matrices with positive definite real part and others without.

In [3]:
from tqdm import tqdm

pd_matrices = []
non_pd_matrices = []

for i in tqdm(range(1000)):
    # A, E, G > 0
    A_val, E_val, G_val = np.random.uniform(0, 5, size=3)

    # sample imaginary parts
    B_val, C_val, D_val, F_val, H_val = np.random.uniform(-5, 5, size=5)

    g_mat_1 = sympy.Matrix([
        [A_val + sympy.I * B_val, C_val + sympy.I * D_val, 0],
        [C_val + sympy.I * D_val, E_val + sympy.I * F_val, 0],
        [0, 0, G_val + sympy.I * H_val]
    ])

    if A_val * E_val > C_val ** 2:
        # it needs to satisfy AE > C^2
        pd_matrices.append(g_mat_1)
    else:
        non_pd_matrices.append(g_mat_1)

100%|██████████| 1000/1000 [00:00<00:00, 1091.89it/s]


In [4]:
test_matrix = pd_matrices[11]
test_matrix

⎡4.07178940774686 + 0.742009009536507⋅ⅈ  0.333211073958247 - 0.92450685312354⋅
⎢                                                                             
⎢0.333211073958247 - 0.92450685312354⋅ⅈ  3.57015932097173 - 3.55142069852302⋅ⅈ
⎢                                                                             
⎣                  0                                       0                  

ⅈ                    0                  ⎤
                                        ⎥
                     0                  ⎥
                                        ⎥
   3.86738022912623 - 4.74161901328391⋅ⅈ⎦

In [5]:
A_val, B_val = test_matrix[0, 0].args
C_val, D_val = test_matrix[0, 1].args
E_val, F_val = test_matrix[1, 1].args
G_val, H_val = test_matrix[2, 2].args

B_val = B_val.args[0]
D_val = D_val.args[0]
F_val = F_val.args[0]
H_val = H_val.args[0]

val_dict = {A: A_val, B: B_val, C: C_val, D: D_val, E: E_val, F: F_val, G: G_val, H: H_val}
val_dict

In [6]:
g_mat = sympy.Matrix([[A + sympy.I * B, C + sympy.I * D, 0], [C + sympy.I * D, E + sympy.I * F, 0], [0, 0, G + sympy.I * H]])
g_mat

⎡A + ⅈ⋅B  C + ⅈ⋅D     0   ⎤
⎢                         ⎥
⎢C + ⅈ⋅D  E + ⅈ⋅F     0   ⎥
⎢                         ⎥
⎣   0        0     G + ⅈ⋅H⎦

In [7]:
g_mat_real = sympy.Matrix([[A, C, 0], [C, E, 0], [0, 0, G]])
g_mat_real

⎡A  C  0⎤
⎢       ⎥
⎢C  E  0⎥
⎢       ⎥
⎣0  0  G⎦

### Diagonalize Real Part

In [8]:
eigenvects_real_sym = g_mat_real.eigenvects()
eigenvects_real_sym

⎡               ⎛                                        ⎡⎡                 __
⎢               ⎜                                        ⎢⎢                ╱  
⎢               ⎜           ________________________     ⎢⎢      A   E   ╲╱  A
⎢⎛      ⎡⎡0⎤⎤⎞  ⎜          ╱  2              2    2      ⎢⎢      ─ + ─ - ─────
⎢⎜      ⎢⎢ ⎥⎥⎟  ⎜A   E   ╲╱  A  - 2⋅A⋅E + 4⋅C  + E       ⎢⎢  E   2   2        
⎢⎜G, 1, ⎢⎢0⎥⎥⎟, ⎜─ + ─ - ───────────────────────────, 1, ⎢⎢- ─ + ─────────────
⎢⎜      ⎢⎢ ⎥⎥⎟  ⎜2   2                2                  ⎢⎢  C                
⎢⎝      ⎣⎣1⎦⎦⎠  ⎜                                        ⎢⎢                   
⎢               ⎜                                        ⎢⎢                   
⎢               ⎜                                        ⎢⎢                   
⎣               ⎝                                        ⎣⎣                   

______________________⎤⎤⎞  ⎛                                        ⎡⎡        
2              2    2 ⎥⎥⎟  ⎜                       

#### Eigenvectors

In [9]:
eig1 = eigenvects_real_sym[2][2][0]
eig1 /= sympy.sqrt(eig1.dot(eig1))

eig2 = eigenvects_real_sym[1][2][0]
eig2 /= sympy.sqrt(eig2.dot(eig2))

eig3 = eigenvects_real_sym[0][2][0]
eig3 /= sympy.sqrt(eig3.dot(eig3))

In [10]:
Q_sym = sympy.Matrix.hstack(eig1, eig2, eig3)
Q_sym.simplify()
Q_sym

⎡                      ________________________                               
⎢                     ╱  2              2    2                                
⎢           A - E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E                           A - E 
⎢──────────────────────────────────────────────────────────  ─────────────────
⎢          ________________________________________________            _______
⎢         ╱                                              2            ╱       
⎢        ╱         ⎛            ________________________⎞            ╱        
⎢       ╱      2   ⎜           ╱  2              2    2 ⎟           ╱      2  
⎢      ╱    4⋅C  + ⎝-A + E - ╲╱  A  - 2⋅A⋅E + 4⋅C  + E  ⎠          ╱    4⋅C  +
⎢C⋅   ╱     ──────────────────────────────────────────────   C⋅   ╱     ──────
⎢    ╱                             2                             ╱            
⎢  ╲╱                             C                            ╲╱             
⎢                                                   

In [11]:
Q_sym.subs(val_dict)

⎡0.894815772471537  -0.446435586995667  0⎤
⎢                                        ⎥
⎢0.446435586995668  0.894815772471537   0⎥
⎢                                        ⎥
⎣        0                  0           1⎦

In [12]:
eigenvects_real_num = sympy.re(test_matrix).eigenvects()
eigenvects_real_num

⎡⎛                     ⎡⎡0.894815772471537⎤⎤⎞  ⎛                     ⎡⎡-0.4464
⎢⎜                     ⎢⎢                 ⎥⎥⎟  ⎜                     ⎢⎢       
⎢⎜4.23803288039502, 1, ⎢⎢0.446435586995668⎥⎥⎟, ⎜3.40391584832358, 1, ⎢⎢0.89481
⎢⎜                     ⎢⎢                 ⎥⎥⎟  ⎜                     ⎢⎢       
⎣⎝                     ⎣⎣        0        ⎦⎦⎠  ⎝                     ⎣⎣       

35586995668⎤⎤⎞  ⎛                     ⎡⎡ 0 ⎤⎤⎞⎤
           ⎥⎥⎟  ⎜                     ⎢⎢   ⎥⎥⎟⎥
5772471537 ⎥⎥⎟, ⎜3.86738022912623, 1, ⎢⎢ 0 ⎥⎥⎟⎥
           ⎥⎥⎟  ⎜                     ⎢⎢   ⎥⎥⎟⎥
 0         ⎦⎦⎠  ⎝                     ⎣⎣1.0⎦⎦⎠⎦

In [13]:
Q_num = sympy.Matrix.hstack(eigenvects_real_num[0][2][0], eigenvects_real_num[1][2][0], eigenvects_real_num[2][2][0])
Q_num

⎡0.894815772471537  -0.446435586995668   0 ⎤
⎢                                          ⎥
⎢0.446435586995668  0.894815772471537    0 ⎥
⎢                                          ⎥
⎣        0                  0           1.0⎦

#### Eigenvalues

In [14]:
lmda_mat_sym = sympy.Matrix([[eigenvects_real_sym[2][0], 0, 0], [0, eigenvects_real_sym[1][0], 0], [0, 0, eigenvects_real_sym[0][0]]])
lmda_mat_sym

⎡           ________________________                                        ⎤
⎢          ╱  2              2    2                                         ⎥
⎢A   E   ╲╱  A  - 2⋅A⋅E + 4⋅C  + E                                          ⎥
⎢─ + ─ + ───────────────────────────                   0                   0⎥
⎢2   2                2                                                     ⎥
⎢                                                                           ⎥
⎢                                                ________________________   ⎥
⎢                                               ╱  2              2    2    ⎥
⎢                                     A   E   ╲╱  A  - 2⋅A⋅E + 4⋅C  + E     ⎥
⎢                 0                   ─ + ─ - ───────────────────────────  0⎥
⎢                                     2   2                2                ⎥
⎢                                                                           ⎥
⎣                 0                                    0        

In [15]:
lmda_mat_sym.subs(val_dict)

⎡4.23803288039502         0                 0        ⎤
⎢                                                    ⎥
⎢       0          3.40391584832358         0        ⎥
⎢                                                    ⎥
⎣       0                 0          3.86738022912623⎦

In [16]:
lmda_mat_num = sympy.Matrix([[eigenvects_real_num[0][0], 0, 0], [0, eigenvects_real_num[1][0], 0], [0, 0, eigenvects_real_num[2][0]]])
lmda_mat_num

⎡4.23803288039502         0                 0        ⎤
⎢                                                    ⎥
⎢       0          3.40391584832358         0        ⎥
⎢                                                    ⎥
⎣       0                 0          3.86738022912623⎦

#### Inverse square root of eigenvalues

In [17]:
P_sym = lmda_mat_sym.inv().applyfunc(sympy.sqrt)
P_sym.simplify()
P_sym

⎡         _____________________________________                               
⎢        ╱                  1                                                 
⎢√2⋅    ╱  ───────────────────────────────────                         0      
⎢      ╱              ________________________                                
⎢     ╱              ╱  2              2    2                                 
⎢   ╲╱     A + E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E                                  
⎢                                                                             
⎢                                                         ____________________
⎢                                                        ╱                  1 
⎢                      0                         √2⋅    ╱  ───────────────────
⎢                                                      ╱              ________
⎢                                                     ╱              ╱  2     
⎢                                                   

In [18]:
P_sym.subs(val_dict).evalf()

⎡0.48575562578588          0                 0        ⎤
⎢                                                     ⎥
⎢       0          0.542014110093643         0        ⎥
⎢                                                     ⎥
⎣       0                  0          0.50850071000715⎦

In [19]:
P_num = lmda_mat_num.inv().applyfunc(sympy.sqrt)
P_num

⎡0.48575562578588          0                 0        ⎤
⎢                                                     ⎥
⎢       0          0.542014110093643         0        ⎥
⎢                                                     ⎥
⎣       0                  0          0.50850071000715⎦

The real part has become the identity.

In [20]:
Z_sym = Q_sym @ P_sym
Z_sym.simplify()
Z_sym

⎡   ⎛           ________________________⎞       ______________________________
⎢   ⎜          ╱  2              2    2 ⎟      ╱                  1           
⎢√2⋅⎝A - E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E  ⎠⋅    ╱  ─────────────────────────────
⎢                                            ╱              __________________
⎢                                           ╱              ╱  2              2
⎢                                         ╲╱     A + E + ╲╱  A  - 2⋅A⋅E + 4⋅C 
⎢─────────────────────────────────────────────────────────────────────────────
⎢                       _______________________________________________       
⎢                      ╱                                             2        
⎢                     ╱         ⎛           ________________________⎞         
⎢                    ╱      2   ⎜          ╱  2              2    2 ⎟         
⎢                   ╱    4⋅C  + ⎝A - E + ╲╱  A  - 2⋅A⋅E + 4⋅C  + E  ⎠         
⎢             C⋅   ╱     ───────────────────────────

In [21]:
Z_sym.subs(val_dict).evalf()

⎡0.434661795519987  -0.24197438739959         0        ⎤
⎢                                                      ⎥
⎢0.216858597934167  0.485002774613916         0        ⎥
⎢                                                      ⎥
⎣        0                  0          0.50850071000715⎦

In [22]:
(Z_sym.T @ g_mat_real @ Z_sym).subs(val_dict)

⎡        1.0           8.88178419700125e-16   0 ⎤
⎢                                               ⎥
⎢8.32667268468867e-16          1.0            0 ⎥
⎢                                               ⎥
⎣         0                     0            1.0⎦

In [23]:
Z_num = Q_num @ P_num
Z_num

⎡0.434661795519987  -0.24197438739959         0        ⎤
⎢                                                      ⎥
⎢0.216858597934167  0.485002774613916         0        ⎥
⎢                                                      ⎥
⎣        0                  0          0.50850071000715⎦

In [24]:
(Q_num @ P_num).T @ sympy.re(test_matrix) @ (Q_num @ P_num)

⎡1.0   0    0 ⎤
⎢             ⎥
⎢ 0   1.0   0 ⎥
⎢             ⎥
⎣ 0    0   1.0⎦

### Diagonalize Imaginary Part

#### Transform imaginary part

In [25]:
g_mat_imag = sympy.Matrix([[B, D, 0], [D, F, 0], [0, 0, H]])
g_mat_imag

⎡B  D  0⎤
⎢       ⎥
⎢D  F  0⎥
⎢       ⎥
⎣0  0  H⎦

In [26]:
g_mat_imag_trans_sym = Z_sym.T @ g_mat_imag @ Z_sym
g_mat_imag_trans_sym.simplify()
g_mat_imag_trans_sym

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                           ⎛ 
⎢                                                                           ⎜ 
⎢                                                                         2⋅⎝2
⎢                                                                         ────
⎢                                                                             
⎢                                                                             
⎢                                                   

In [27]:
g_mat_imag_trans_sym.subs(val_dict).evalf()

⎡-0.201114867178124  -0.597954669867259          0        ⎤
⎢                                                         ⎥
⎢-0.597954669867259  -0.574949662787112          0        ⎥
⎢                                                         ⎥
⎣        0                   0           -1.22605452072531⎦

In [28]:
g_mat_imag_trans_num = Z_num.T @ sympy.im(test_matrix) @ Z_num
g_mat_imag_trans_num

⎡-0.201114867178124  -0.597954669867259          0        ⎤
⎢                                                         ⎥
⎢-0.597954669867259  -0.574949662787111          0        ⎥
⎢                                                         ⎥
⎣        0                   0           -1.22605452072531⎦

#### Eigenvectors

In [29]:
eigenvects_imag_sym = g_mat_imag_trans_sym.eigenvects()
eigenvects_imag_sym

⎡               ⎛                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                                                             
⎢               ⎜                       ______________________________________
⎢⎛      ⎡⎡0⎤⎤⎞  ⎜                      ╱  2  2                                
⎢⎜H     ⎢⎢ ⎥⎥⎟  ⎜A⋅F + B⋅E - 2⋅C⋅D   ╲╱  A ⋅F  - 2⋅A⋅B⋅E⋅F - 4⋅A⋅C⋅D⋅F + 4⋅A⋅D
⎢⎜─, 1, ⎢⎢0⎥⎥⎟, ⎜───────────────── - ─────────────────────────────────────────
⎢⎜G     ⎢⎢ ⎥⎥⎟  ⎜     ⎛       2⎞                                       ⎛      
⎢⎝      ⎣⎣1⎦⎦⎠  ⎜   2⋅⎝A⋅E - C ⎠                    

In [30]:
eig_imag_1 = eigenvects_imag_sym[2][2][0]
eig_imag_1 /= sympy.sqrt(eig_imag_1.dot(eig_imag_1))

eig_imag_2 = eigenvects_imag_sym[1][2][0]
eig_imag_2 /= sympy.sqrt(eig_imag_2.dot(eig_imag_2))

eig_imag_3 = eigenvects_imag_sym[0][2][0]
eig_imag_3 /= sympy.sqrt(eig_imag_3.dot(eig_imag_3))

In [31]:
Q2_sym = sympy.Matrix.hstack(eig_imag_1, eig_imag_2, eig_imag_3)

In [32]:
Q2_sym.subs(val_dict).evalf()

⎡-0.80571621739337  0.592301761798726   0 ⎤
⎢                                         ⎥
⎢0.592301761798805  0.805716217392734   0 ⎥
⎢                                         ⎥
⎣        0                  0          1.0⎦

In [33]:
eigenvects_imag_num = g_mat_imag_trans_num.eigenvects()
eigenvects_imag_num

⎡⎛                      ⎡⎡ 0.80571621739297 ⎤⎤⎞  ⎛                      ⎡⎡0.59
⎢⎜                      ⎢⎢                  ⎥⎥⎟  ⎜                      ⎢⎢    
⎢⎜0.238456283051635, 1, ⎢⎢-0.592301761798802⎥⎥⎟, ⎜-1.01452081301687, 1, ⎢⎢0.80
⎢⎜                      ⎢⎢                  ⎥⎥⎟  ⎜                      ⎢⎢    
⎣⎝                      ⎣⎣        0         ⎦⎦⎠  ⎝                      ⎣⎣    

2301761798802⎤⎤⎞  ⎛                      ⎡⎡ 0 ⎤⎤⎞⎤
             ⎥⎥⎟  ⎜                      ⎢⎢   ⎥⎥⎟⎥
571621739297 ⎥⎥⎟, ⎜-1.22605452072531, 1, ⎢⎢ 0 ⎥⎥⎟⎥
             ⎥⎥⎟  ⎜                      ⎢⎢   ⎥⎥⎟⎥
    0        ⎦⎦⎠  ⎝                      ⎣⎣1.0⎦⎦⎠⎦

In [34]:
Q2_num = sympy.Matrix.hstack(eigenvects_imag_num[1][2][0], eigenvects_imag_num[0][2][0], eigenvects_imag_num[2][2][0])
Q2_num

⎡0.592301761798802   0.80571621739297    0 ⎤
⎢                                          ⎥
⎢0.80571621739297   -0.592301761798802   0 ⎥
⎢                                          ⎥
⎣        0                  0           1.0⎦

#### Eigenvalues

In [35]:
lmda_mat_sym_2 = sympy.Matrix([[eigenvects_imag_sym[2][0], 0, 0], [0, eigenvects_imag_sym[1][0], 0], [0, 0, eigenvects_imag_sym[0][0]]])
lmda_mat_sym_2

⎡                       ______________________________________________________
⎢                      ╱  2  2                                2      2  2     
⎢A⋅F + B⋅E - 2⋅C⋅D   ╲╱  A ⋅F  - 2⋅A⋅B⋅E⋅F - 4⋅A⋅C⋅D⋅F + 4⋅A⋅D ⋅E + B ⋅E  + 4⋅
⎢───────────────── + ─────────────────────────────────────────────────────────
⎢     ⎛       2⎞                                       ⎛       2⎞             
⎢   2⋅⎝A⋅E - C ⎠                                     2⋅⎝A⋅E - C ⎠             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                               0                             
⎢                                                                             
⎢                                                   

In [51]:
lmda_mat_sym_2[0, 0].args[0].args[2].args[0]

In [36]:
lmda_mat_sym_2.subs(val_dict)

⎡0.238456283051635          0                  0        ⎤
⎢                                                       ⎥
⎢        0          -1.01452081301687          0        ⎥
⎢                                                       ⎥
⎣        0                  0          -1.22605452072531⎦

In [37]:
lmda_mat_num_2 = sympy.Matrix([[eigenvects_imag_num[1][0], 0, 0], [0, eigenvects_imag_num[0][0], 0], [0, 0, eigenvects_imag_num[2][0]]])
lmda_mat_num_2

⎡-1.01452081301687          0                  0        ⎤
⎢                                                       ⎥
⎢        0          0.238456283051635          0        ⎥
⎢                                                       ⎥
⎣        0                  0          -1.22605452072531⎦

### Final Transformation

In [38]:
T_sym = Q_sym @ P_sym @ Q2_sym

In [39]:
T_sym.subs(val_dict).evalf()

⎡-0.493535913698738  0.0624882591515641         0        ⎤
⎢                                                        ⎥
⎢0.112541508644385   0.519220330604514          0        ⎥
⎢                                                        ⎥
⎣        0                   0           0.50850071000715⎦

In [40]:
(T_sym.T @ g_mat @ T_sym).subs(val_dict).evalf()

⎡    1.00000000000065 + 0.238456283051789⋅ⅈ      -3.12847096266245e-13 + 2.183
⎢                                                                             
⎢-3.12847096266245e-13 + 2.18386447012764e-13⋅ⅈ      0.999999999999531 - 1.014
⎢                                                                             
⎣                      0                                               0      

86447012764e-13⋅ⅈ             0            ⎤
                                           ⎥
52081301639⋅ⅈ                 0            ⎥
                                           ⎥
                   1.0 - 1.22605452072531⋅ⅈ⎦

In [41]:
T_num = Q_num @ P_num @ Q2_num
T_num

⎡0.0624882591515398  0.493535913698563         0        ⎤
⎢                                                       ⎥
⎢0.519220330604645   -0.11254150864447         0        ⎥
⎢                                                       ⎥
⎣        0                   0          0.50850071000715⎦

In [42]:
T_num.T @ test_matrix @ T_num

⎡          1.0 - 1.01452081301687⋅ⅈ               1.11022302462516e-16        
⎢                                                                             
⎢1.11022302462516e-16 + 1.04083408558608e-17⋅ⅈ  1.0 + 0.238456283051635⋅ⅈ     
⎢                                                                             
⎣                      0                                    0              1.0

        0            ⎤
                     ⎥
        0            ⎥
                     ⎥
 - 1.22605452072531⋅ⅈ⎦